In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key="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",
    # other params...
)

response = llm.invoke("the first man to walk on moon was ...")
print(response.content)

The first man to walk on the moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first person to set foot on the moon.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/retail-associate-pt-livermore/job/R-64713")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.






















Retail associate, PT - Livermore










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Me

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template(
    """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_template | llm
res = chain_extract.invoke(input={'page_data': page_data})
type(res.content)

str

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json = JsonOutputParser()
json_res = json.parse(res.content)
json_res

{'role': 'Retail Associate',
 'experience': 'Part-time',
 'skills': ['Customer service',
  'Communication',
  'Teamwork',
  'Adaptability',
  'Basic math skills',
  'Ability to work in a fast-paced environment'],
 'description': "We're looking for a Retail Associate to join our team in Livermore, California. As a Retail Associate, you will be the face of NIKE, providing excellent customer service and helping customers make informed purchasing decisions. You will work in a fast-paced, high-volume environment and be expected to be knowledgeable about NIKE products and services. You will also be responsible for operating a cash register, shipping and receiving duties, stocking products, and building visual displays."}

In [14]:
type(json_res)

dict

In [8]:
import pandas as pd

df = pd.read_csv('my_portfolio.csv')
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://dev-portfolio-one-bay.vercel.app/
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://github.com/Archesus/django-todo-cicd
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://github.com/Archesus/ak-uber
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [14]:
# links=collection.query(query_texts=["Experience in React", "Expertise in Python"],n_results=2).get('metadatas', [])
links=collection.query(query_texts=job['skills'],n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/android-tv-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/android-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/android-portfolio'}]]

In [13]:
job = json_res
job['skills']

['Customer service',
 'Communication',
 'Teamwork',
 'Adaptability',
 'Basic math skills',
 'Ability to work in a fast-paced environment']

In [15]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Anurag, a junior technical associate at Galaxy Office Automation Pvt Ltd. Galaxy is one of the largest IT Infrastructure solution providers in India, offering cutting edge Data Center, Cloud, Networking, Cyber Security and Modern Workplaces solutions 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Galaxy 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Galaxy's portfolio: {link_list}
        Remember you are Anurag, JTA at Galaxy. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_mail=prompt_email | llm
res = chain_mail.invoke({'job_description': str(job), 'link_list': links})
print(res.content)

Subject: Empowering NIKE's Retail Operations with Galaxy's Expertise

Dear Hiring Manager,

I came across the job posting for a Retail Associate at NIKE in Livermore, California, and I was impressed by the company's commitment to providing excellent customer service. As a Junior Technical Associate at Galaxy Office Automation Pvt Ltd, I believe our company's expertise in IT infrastructure solutions can support NIKE's retail operations and enhance the overall customer experience.

At Galaxy, we have a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our expertise in Data Center, Cloud, Networking, Cyber Security, and Modern Workplaces solutions can help NIKE streamline its retail operations, improve inventory management, and enhance customer engagement.

I would like to highlight some of our relevant portfolio links that demonstrate our capabilities:

* https://example.